Megan Sisson
DSC 680 Applied Data Science
Project 3


Customer Review using Vader to understand the sentiment of text.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

import collections


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

C:\Users\mksis\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#nltk.download('vader_lexicon')

In [ ]:
df = pd.read_csv('/Users/mksis/Documents/Data Science/680 Applied DS/Reviews.csv')
df.head()

In [ ]:
#Column names
df.columns

In [ ]:
# Checking for null values
df.isnull().sum()

In [ ]:
#Seeing row for null value from reviewerName
df[df['reviewerName'].isna() == True]

In [ ]:
#Grouping reviews by reviewTime to see the frequency of reviews each day.
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
df_group = df.groupby('reviewTime').count()

df_group.head()

In [ ]:
#Number of reviews by time
df_group.plot(y = 'reviewerID', use_index = True, ylabel = 'Count', title = 'Count of Reviews Over Time')

In [ ]:
# Count Review rating [overall]
rating = df['overall'].value_counts()
rating

In [ ]:
#Graph for Ratings
rating.plot.bar(title = 'Rating Counts')
plt.show()

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
#Convert reviewText2 to string
df['reviewText'] = df['reviewText'].apply(str)

In [ ]:
#Clean the reviewText
#lowercase text
df['reviewText2'] = df['reviewText'].str.lower()
#remove punctuation
df['reviewText2'] = df['reviewText2'].str.replace('[^\w\s]', '')

df.head()

In [ ]:
#sentiment analyzer. Adding polarity columns
df['polarity'] = df['reviewText2'].apply(lambda x: sia.polarity_scores(x))

df

In [ ]:
#transform polarity column into columns for negative, neutral, positive, and compound

df[['neg', 'neu', 'pos', 'compound']] = df['polarity'].apply(pd.Series)
df

In [ ]:
#Visualize compound score compared to review score
ax = sb.barplot(data = df, x = 'overall', y = 'compound')
ax.set_title('Compound Score by Amazon Overall Score')
plt.show()
#This plot makes sense, as the compound score is low, it should also correlate to a low overall score
# and a higher compound score should correlate to a higher overall score

In [ ]:
#Visualize overall score to the negative, neutral, and positive scores
fig, axs = plt.subplots(1, 3, figsize = (15, 5))
sb.barplot(data = df, x = 'overall', y = 'pos', ax=axs[0])
sb.barplot(data = df, x = 'overall', y = 'neu', ax=axs[1])
sb.barplot(data = df, x = 'overall', y = 'neg', ax=axs[2])
axs[0].set_title('Positive')
axs[1].set_title('Neutral')
axs[2].set_title('Negative')
#Confirms what we hope to see
#Positive polarity increases as overall score increases
#Negative polarity increases as overall score increases

In [ ]:
#Scatter plot to see relationship between overall score and compound score
df.plot.scatter(x = 'overall', y = 'neg')

In [ ]:
#Dropping columns not needed for model
df2 = df.drop(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'summary', 'unixReviewTime', 'reviewTime', 'day_diff',
       'helpful_yes', 'total_vote', 'reviewText2', 'polarity'], axis = 1)
df2.head()

In [ ]:
#Converting 'overall' to string
df2['overall'] = df2['overall'].astype(str)

In [ ]:
#Split data into X(independent) and y(dependent)
X = df2.drop('overall', axis = 1)
y = df2.overall
print("X shape: ", X.shape)
print('y shape: ', y.shape)

In [ ]:
#Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 42)
print('Training shape: ', X_train.shape)
print('Testing shape: ', X_test.shape)


In [ ]:
#Fitting the model Random Forest Classification
classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

In [ ]:
#Make Predictions
y_pred = classifier.predict(X_test)
y_pred

In [ ]:
#Graph Actual vs Predicted
plt.scatter(y_test, y_pred)
plt.xlabel('Actual overall score')
plt.ylabel('Predicted overall score')
plt.title('Actual vs Predicted overall score')
plt.show()

In [ ]:
#Accuracy Score
acc = accuracy_score(y_test, y_pred)
print("Accuracy score: ", acc)

In [ ]:
#Confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
#Confusion Matrix Heatmap
sb.heatmap(cm, annot = True, fmt = 'd', xticklabels = ['1', '2', '3', '4', '5'], 
           yticklabels = ['1', '2', '3', '4', '5'], cmap = plt.cm.Greens)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Random Forest Confusion Matrix')
plt.show()

In [ ]:
#Classification Report
report = classification_report(y_test, y_pred)
print(report)